# Dialect Conversion Model

convert text between UK and US dialects.

# Data Preparation

In [1]:
! pip install transformers datasets

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
neo-api-client 1.2.0 requires requests==2.25.1, but you have requests 2.32.3 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.2 which is incompatible.


* Load Dataset

In [2]:
import pandas as pd
import re

In [3]:
file=r"D:\Data centr\Exel_data\int data\CozmoX Assignment Dataset.csv"
data=pd.read_csv(file)
data

,input_text,target_text
0,I CoLoUr 🎨 the centre of my favourite book.,I color the center of my favorite book.
1,He is travelling ✈️ to the THEATRE.,He is traveling to the theater.
2,I have a flat near the lift.,I have an apartment near the elevator.
3,I have a flat near the lift.,I have an apartment near the elevator.
4,The PROGRAMME 🗓️ will start at 6 O'CLOCK.,The program will start at 6 o'clock.
...,...,...
91,The theatre's performance was breathtaking.,The theater's performance was breathtaking.
92,Her behaviour has been commendable.,Her behavior has been commendable.
93,The cheque was never received.,The check was never received.
94,The aeroplane ✈️ took off on time.,The airplane took off on time.


In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   input_text   96 non-null     object
 1   target_text  96 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB
None


In [5]:
data.isnull().sum()

input_text     0
target_text    0
dtype: int64

* Normalization

In [6]:
import unicodedata

In [7]:
def clean_text(text):
  text=unicodedata.normalize("NFKC", text) # Normalize the char
  text=text.lower()
  text=re.sub(r"[^\w\s]", "", text)  # remove special characters & Emojis
  text=re.sub(r"\s+", " ", text)  # remove extra spaces
  return text

Data Cleaning

In [8]:
data["input_text"] = data["input_text"].apply(clean_text)
data["target_text"] = data["target_text"].apply(clean_text)

In [9]:
print(data.head())

                                 input_text   
0  i colour the centre of my favourite book  \
1           he is travelling to the theatre   
2               i have a flat near the lift   
3              i have a flat near the lift    
4      the programme will start at 6 oclock   

                              target_text  
0  i color the center of my favorite book  
1          he is traveling to the theater  
2   i have an apartment near the elevator  
3   i have an apartment near the elevator  
4      the program will start at 6 oclock  


In [10]:
data.to_csv("processed_cozmoX.csv", index=False)
print("\n saved")


 saved


# Model Selection- T5
Pretrained on Text Transformation.

Best for small dataset with accuracy.

In [11]:
import tensorflow as tf
from transformers import T5Tokenizer, T5ForConditionalGeneration, TFAutoModelForSeq2SeqLM

c:\Users\himan\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Tokenization- breaking text into smaller units (tokens)

In [12]:
tokanizer=T5Tokenizer.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
model_t5=TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [14]:
def tokenize_data(sentence):
  input_text = "translate UK to US: " + sentence["input_text"]  # Task prefix
  target_text = sentence["target_text"]

  tokenized_inputs = tokanizer(input_text, padding="max_length", truncation=True, max_length=128, return_tensors="tf")
  tokenized_targets=tokanizer(target_text, padding="max_length", truncation=True, max_length=128, return_tensors="tf")

  return{
      "input_ids": tokenized_inputs["input_ids"][0],  # Extract tensors
      "attention_mask": tokenized_inputs["attention_mask"][0],
      "labels": tokenized_targets["input_ids"][0],
      }

token_dataset=data.apply(tokenize_data, axis=1)

Convert the tokenized dataset to a TensorFlow dataset

In [15]:
def to_tf(tokenized_data):
  input_ids=[]
  attention_masks=[]
  labels=[]

  for item in tokenized_data:
    input_ids.append(item["input_ids"])
    attention_masks.append(item["attention_mask"])
    labels.append(item["labels"])

  return tf.data.Dataset.from_tensor_slices(({
      "input_ids": input_ids,
      "attention_mask": attention_masks
  },
      tf.stack(labels)
  ))
  tf_dataset=to_tf(token_dataset)

# Model Training

In [16]:
lr_sechedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_sechedule)
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [18]:
epochs=3
tf_dataset=to_tf(token_dataset).batch(1)

for epoch in range(epochs):
  for batch in tf_dataset:
    inputs, labels = batch

    with tf.GradientTape() as tape:
      # Forward pass
      outputs = model_t5(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
      loss = outputs.loss

      # Backward pass
      gradients = tape.gradient(loss, model_t5.trainable_variables)
      optimizer.apply_gradients(zip(gradients, model_t5.trainable_variables))

      print(f"Loss: {loss.numpy()}")

  print(f"Completed epoch {epoch + 1}")

Loss: [12.706329]
Loss: [2.2442045]
Loss: [0.25050882]
Loss: [0.3531435]
Loss: [0.39835292]
Loss: [0.24258265]
Loss: [0.22039552]
Loss: [0.25458455]
Loss: [0.1920833]
Loss: [0.11431481]
Loss: [0.13166723]
Loss: [0.0862973]
Loss: [0.10901935]
Loss: [0.11264656]
Loss: [0.9105618]
Loss: [0.17528506]
Loss: [0.21915969]
Loss: [0.24879193]
Loss: [0.23987854]
Loss: [0.20293619]
Loss: [0.22588362]
Loss: [0.23429261]
Loss: [0.24289829]
Loss: [0.19904365]
Loss: [0.24879585]
Loss: [0.16451147]
Loss: [0.20556447]
Loss: [0.1878223]
Loss: [0.14254296]
Loss: [0.10083942]
Loss: [0.22203428]
Loss: [0.14729905]
Loss: [0.10394251]
Loss: [0.1199896]
Loss: [0.08560469]
Loss: [0.08905366]
Loss: [0.09182685]
Loss: [0.12435038]
Loss: [0.05666936]
Loss: [0.09219187]
Loss: [0.19628417]
Loss: [0.07823987]
Loss: [0.06094883]
Loss: [0.10114169]
Loss: [0.10299458]
Loss: [0.05737923]
Loss: [0.07074274]
Loss: [0.05352058]
Loss: [0.14460175]
Loss: [0.04164637]
Loss: [0.06234517]
Loss: [0.04528929]
Loss: [0.11718763]
L

# Model Evaluation

In [22]:
from evaluate import load

In [26]:
bleu_metric = load("bleu")
rouge_metric = load("rouge")

ImportError: To be able to use evaluate-metric/rouge, you need to install the following dependencies['rouge_score'] using 'pip install rouge_score' for instance'

In [ ]:
epochs = 3
batch_size = 1
tf_dataset = to_tf(token_dataset).batch(batch_size)

# Graph Visualization

# Model Deployment